In [6]:
!pip install streamlit pyngrok joblib

In [3]:
%%writefile credit_card_fraud_model.py
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

df = pd.read_csv("creditcard.csv")
scaler = StandardScaler()
df[['Time', 'Amount']] = scaler.fit_transform(df[['Time', 'Amount']])

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

joblib.dump(model, "fraud_model.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Model and scaler saved!")

Overwriting credit_card_fraud_model.py


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

model = joblib.load("fraud_model.pkl")
scaler = joblib.load("scaler.pkl")

st.title("Credit Card Fraud Detection")

file = st.file_uploader("Upload transaction CSV", type="csv")
if file:
    df = pd.read_csv(file)
    if 'Time' in df.columns and 'Amount' in df.columns:
        df[['Time', 'Amount']] = scaler.transform(df[['Time', 'Amount']])
    if 'Class' in df.columns:
        df = df.drop('Class', axis=1)
    preds = model.predict(df)
    probs = model.predict_proba(df)[:,1]
    df['Prediction'] = preds
    df['Fraud_Probability'] = probs.round(3)
    st.dataframe(df[['Prediction','Fraud_Probability']].head())
    st.download_button("Download CSV", df.to_csv(index=False).encode(), "results.csv", "text/csv")

Writing app.py
